<a href="https://colab.research.google.com/github/catekat16/TechTogetherTeam/blob/ketki/ML%20Model/MB_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir .kaggle
!ls

sample_data


In [2]:
import json
token = {"username":"dazzykin","key":"8e623a44cc7443bae9c0b42ca5d093c8"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d datasnaek/mbti-type

 37% 9.00M/24.4M [00:00<00:01, 9.26MB/s]
100% 24.4M/24.4M [00:01<00:00, 24.0MB/s]


In [7]:
%cd {/content}/datasets/datasnaek/mbti-type

/content/{/content}/datasets/datasnaek/mbti-type


In [8]:
!unzip \*.zip

Archive:  mbti-type.zip
  inflating: mbti_1.csv              


In [9]:
pip install contractions

     |████████████████████████████████| 317kB 8.4MB/s 
     |████████████████████████████████| 245kB 16.8MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81701 sha256=dd371b2b141e1987402922538d3bf1173d738594c2870aa68d2693730b92ce0d
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [10]:
import nltk
import contractions
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords, wordnet
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf

In [12]:
corpus = pd.read_csv("mbti_1.csv")
corpus.head(10)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...
5,INTJ,'18/37 @.@|||Science is not perfect. No scien...
6,INFJ,"'No, I can't draw on my own nails (haha). Thos..."
7,INTJ,'I tend to build up a collection of things on ...
8,INFJ,"I'm not sure, that's a good question. The dist..."
9,INTP,'https://www.youtube.com/watch?v=w8-egj0y8Qs||...


In [13]:
!pip install tweet-preprocessor

In [14]:
#Remove URLs
import preprocessor as p
corpus['posts'] = corpus['posts'].apply(p.clean)

In [18]:
corpus.head(30)

,type,posts
0,INFJ,' and intj moments sportscenter not top ten pl...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,"'Good one _____ course, to which I say I know;..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the oth..."
4,ENTJ,'You're fired.|||That's another silly misconce...
5,INTJ,'18/37 .|||Science is not perfect. No scientis...
6,INFJ,"'No, I can't draw on my own nails (haha). Thos..."
7,INTJ,'I tend to build up a collection of things on ...
8,INFJ,"I'm not sure, that's a good question. The dist..."
9,INTP,' in this position where I have to actually le...


In [19]:
#Make Everything Lowercase
corpus['posts'] =  corpus["posts"].str.lower()
corpus['type'] =  corpus["type"].str.lower()

In [20]:
# Remove Punctuation and Numbers
def remove_punctuations_numbers(text):
    punc = string.punctuation +'1234567890' 
    #print(punc)
    for punctuation in punc:
        text = text.replace(punctuation, '')
    return text

In [21]:
corpus['posts'] =  corpus["posts"].apply(remove_punctuations_numbers)
corpus.head()

,type,posts
0,infj,and intj moments sportscenter not top ten pla...
1,entp,im finding the lack of me in these posts very ...
2,intp,good one course to which i say i know thats m...
3,intj,dear intp i enjoyed our conversation the other...
4,entj,youre firedthats another silly misconception t...


In [22]:
#Remove Word contractions
corpus['posts'] =  corpus["posts"].apply(lambda x: [contractions.fix(word) for word in x.split()])
corpus.head()

,type,posts
0,infj,"[and, intj, moments, sportscenter, not, top, t..."
1,entp,"[I am, finding, the, lack, of, me, in, these, ..."
2,intp,"[good, one, course, to, which, i, say, i, know..."
3,intj,"[dear, intp, i, enjoyed, our, conversation, th..."
4,entj,"[you are, firedthats, another, silly, misconce..."


In [23]:
labels=list(corpus.type.unique())

In [24]:
stop_words=set(stopwords.words('english')+labels)

In [25]:
texts = corpus['posts'].copy()
labels = corpus['type'].copy()
#texts = [[word for word in text if word not in stop_words] for text in texts]

In [26]:
vocab_length=10000

In [27]:
#Remove Stop Words
corpus['posts'] =  corpus["posts"].apply(lambda x: [word for word in x if word not in stop_words])
corpus.head()

,type,posts
0,infj,"[moments, sportscenter, top, ten, plays, prank..."
1,entp,"[I am, finding, lack, posts, alarmingsex, bori..."
2,intp,"[good, one, course, say, know, that is, blessi..."
3,intj,"[dear, enjoyed, conversation, day, esoteric, g..."
4,entj,"[you are, firedthats, another, silly, misconce..."


In [28]:
#Clean body Str:
corpus['clean_body_str'] = [' '.join(map(str, l)) for l in corpus['posts']]
corpus.head()

,type,posts,clean_body_str
0,infj,"[moments, sportscenter, top, ten, plays, prank...",moments sportscenter top ten plays prankswhat ...
1,entp,"[I am, finding, lack, posts, alarmingsex, bori...",I am finding lack posts alarmingsex boring pos...
2,intp,"[good, one, course, say, know, that is, blessi...",good one course say know that is blessing curs...
3,intj,"[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...
4,entj,"[you are, firedthats, another, silly, misconce...",you are firedthats another silly misconception...


In [29]:
#Tokenize
corpus['tokenized'] = corpus['clean_body_str'].apply(word_tokenize)
corpus.head()

,type,posts,clean_body_str,tokenized
0,infj,"[moments, sportscenter, top, ten, plays, prank...",moments sportscenter top ten plays prankswhat ...,"[moments, sportscenter, top, ten, plays, prank..."
1,entp,"[I am, finding, lack, posts, alarmingsex, bori...",I am finding lack posts alarmingsex boring pos...,"[I, am, finding, lack, posts, alarmingsex, bor..."
2,intp,"[good, one, course, say, know, that is, blessi...",good one course say know that is blessing curs...,"[good, one, course, say, know, that, is, bless..."
3,intj,"[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...,"[dear, enjoyed, conversation, day, esoteric, g..."
4,entj,"[you are, firedthats, another, silly, misconce...",you are firedthats another silly misconception...,"[you, are, firedthats, another, silly, misconc..."


In [30]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [31]:
#Tagging Parts of Speech in the tokenized 
corpus['pos_tags'] = corpus['tokenized'].apply(nltk.tag.pos_tag)
corpus.head()

,type,posts,clean_body_str,tokenized,pos_tags
0,infj,"[moments, sportscenter, top, ten, plays, prank...",moments sportscenter top ten plays prankswhat ...,"[moments, sportscenter, top, ten, plays, prank...","[(moments, NNS), (sportscenter, VBP), (top, JJ..."
1,entp,"[I am, finding, lack, posts, alarmingsex, bori...",I am finding lack posts alarmingsex boring pos...,"[I, am, finding, lack, posts, alarmingsex, bor...","[(I, PRP), (am, VBP), (finding, VBG), (lack, N..."
2,intp,"[good, one, course, say, know, that is, blessi...",good one course say know that is blessing curs...,"[good, one, course, say, know, that, is, bless...","[(good, JJ), (one, CD), (course, NN), (say, VB..."
3,intj,"[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...,"[dear, enjoyed, conversation, day, esoteric, g...","[(dear, JJ), (enjoyed, JJ), (conversation, NN)..."
4,entj,"[you are, firedthats, another, silly, misconce...",you are firedthats another silly misconception...,"[you, are, firedthats, another, silly, misconc...","[(you, PRP), (are, VBP), (firedthats, JJ), (an..."


In [32]:
corpus['wordnet_pos'] = corpus['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
corpus.head()

,type,posts,clean_body_str,tokenized,pos_tags,wordnet_pos
0,infj,"[moments, sportscenter, top, ten, plays, prank...",moments sportscenter top ten plays prankswhat ...,"[moments, sportscenter, top, ten, plays, prank...","[(moments, NNS), (sportscenter, VBP), (top, JJ...","[(moments, n), (sportscenter, v), (top, a), (t..."
1,entp,"[I am, finding, lack, posts, alarmingsex, bori...",I am finding lack posts alarmingsex boring pos...,"[I, am, finding, lack, posts, alarmingsex, bor...","[(I, PRP), (am, VBP), (finding, VBG), (lack, N...","[(I, n), (am, v), (finding, v), (lack, n), (po..."
2,intp,"[good, one, course, say, know, that is, blessi...",good one course say know that is blessing curs...,"[good, one, course, say, know, that, is, bless...","[(good, JJ), (one, CD), (course, NN), (say, VB...","[(good, a), (one, n), (course, n), (say, v), (..."
3,intj,"[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...,"[dear, enjoyed, conversation, day, esoteric, g...","[(dear, JJ), (enjoyed, JJ), (conversation, NN)...","[(dear, a), (enjoyed, a), (conversation, n), (..."
4,entj,"[you are, firedthats, another, silly, misconce...",you are firedthats another silly misconception...,"[you, are, firedthats, another, silly, misconc...","[(you, PRP), (are, VBP), (firedthats, JJ), (an...","[(you, n), (are, v), (firedthats, a), (another..."


In [33]:
print(corpus['clean_body_str'][0])

moments sportscenter top ten plays prankswhat lifechanging experience life repeat todaymay perc experience immerse youthe last thing friend posted facebook committing suicide next day rest peace sorry hear distress natural relationship perfection time every moment existence try figure hard times times growth welcome stuff game set matchprozac wellbrutin least thirty minutes moving legs do not mean moving sitting desk chair weed moderation maybe try edibles healthier alternativebasically come three items you have determined type whichever types want would likely use given types cognitive functions whatnot left byall things moderation sims indeed video game good one note good one somewhat subjective completely promoting death given simdear favorite video games growing current favorite video games cool appears late adtheres someone everyonewait thought confidence good thingi cherish time solitude bc revel within inner world whereas time id workin enjoy time do not worry people always arou

In [34]:
#Lemmatization
wnl = WordNetLemmatizer()
corpus['lemmatized'] = corpus['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
corpus.head()

,type,posts,clean_body_str,tokenized,pos_tags,wordnet_pos,lemmatized
0,infj,"[moments, sportscenter, top, ten, plays, prank...",moments sportscenter top ten plays prankswhat ...,"[moments, sportscenter, top, ten, plays, prank...","[(moments, NNS), (sportscenter, VBP), (top, JJ...","[(moments, n), (sportscenter, v), (top, a), (t...","[moment, sportscenter, top, ten, play, pranksw..."
1,entp,"[I am, finding, lack, posts, alarmingsex, bori...",I am finding lack posts alarmingsex boring pos...,"[I, am, finding, lack, posts, alarmingsex, bor...","[(I, PRP), (am, VBP), (finding, VBG), (lack, N...","[(I, n), (am, v), (finding, v), (lack, n), (po...","[I, be, find, lack, post, alarmingsex, boring,..."
2,intp,"[good, one, course, say, know, that is, blessi...",good one course say know that is blessing curs...,"[good, one, course, say, know, that, is, bless...","[(good, JJ), (one, CD), (course, NN), (say, VB...","[(good, a), (one, n), (course, n), (say, v), (...","[good, one, course, say, know, that, be, bless..."
3,intj,"[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...,"[dear, enjoyed, conversation, day, esoteric, g...","[(dear, JJ), (enjoyed, JJ), (conversation, NN)...","[(dear, a), (enjoyed, a), (conversation, n), (...","[dear, enjoyed, conversation, day, esoteric, g..."
4,entj,"[you are, firedthats, another, silly, misconce...",you are firedthats another silly misconception...,"[you, are, firedthats, another, silly, misconc...","[(you, PRP), (are, VBP), (firedthats, JJ), (an...","[(you, n), (are, v), (firedthats, a), (another...","[you, be, firedthats, another, silly, misconce..."


In [35]:
#Add Lemma Str column
corpus['lemma_str'] = [' '.join(map(str,l)) for l in corpus['lemmatized']]
corpus.head()

,type,posts,clean_body_str,tokenized,pos_tags,wordnet_pos,lemmatized,lemma_str
0,infj,"[moments, sportscenter, top, ten, plays, prank...",moments sportscenter top ten plays prankswhat ...,"[moments, sportscenter, top, ten, plays, prank...","[(moments, NNS), (sportscenter, VBP), (top, JJ...","[(moments, n), (sportscenter, v), (top, a), (t...","[moment, sportscenter, top, ten, play, pranksw...",moment sportscenter top ten play prankswhat li...
1,entp,"[I am, finding, lack, posts, alarmingsex, bori...",I am finding lack posts alarmingsex boring pos...,"[I, am, finding, lack, posts, alarmingsex, bor...","[(I, PRP), (am, VBP), (finding, VBG), (lack, N...","[(I, n), (am, v), (finding, v), (lack, n), (po...","[I, be, find, lack, post, alarmingsex, boring,...",I be find lack post alarmingsex boring positio...
2,intp,"[good, one, course, say, know, that is, blessi...",good one course say know that is blessing curs...,"[good, one, course, say, know, that, is, bless...","[(good, JJ), (one, CD), (course, NN), (say, VB...","[(good, a), (one, n), (course, n), (say, v), (...","[good, one, course, say, know, that, be, bless...",good one course say know that be bless cursedo...
3,intj,"[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...,"[dear, enjoyed, conversation, day, esoteric, g...","[(dear, JJ), (enjoyed, JJ), (conversation, NN)...","[(dear, a), (enjoyed, a), (conversation, n), (...","[dear, enjoyed, conversation, day, esoteric, g...",dear enjoyed conversation day esoteric gabbing...
4,entj,"[you are, firedthats, another, silly, misconce...",you are firedthats another silly misconception...,"[you, are, firedthats, another, silly, misconc...","[(you, PRP), (are, VBP), (firedthats, JJ), (an...","[(you, n), (are, v), (firedthats, a), (another...","[you, be, firedthats, another, silly, misconce...",you be firedthats another silly misconception ...


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect=None
count_vect = CountVectorizer(lowercase=False)
X_train_counts = count_vect.fit_transform(corpus['lemma_str'].tolist())
X_train_counts.shape

KeyError: ignored

In [37]:
Train_X = None
Train_Y = None
Test_X = None
Test_Y = None
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(corpus['lemma_str'],corpus['type'],test_size=0.2)

In [55]:
Tfidf_vect=None
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(corpus['lemma_str'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [57]:
X_train = Train_X_Tfidf
Y_train = Train_Y
X_test = Test_X_Tfidf
Y_test = Test_Y

In [61]:
print(X_train.shape, Y_train.shape , Y_test.shape, X_test.shape )

(6940, 5000) (6940,) (1735,) (1735, 5000)


In [ ]:
import matplotlib.pyplot as plt

plt.hist(Y_test)

In [59]:
models = [
    svm.SVC(C=1.0, kernel='linear', degree=10, gamma='auto'),
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0)
]

In [60]:
CV = 3
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
y_preds=[]
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, X_train, Y_train, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy*100))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [63]:
cv_df.groupby('model_name').accuracy.mean()

model_name
LinearSVC                 49.855775
LogisticRegression        48.659780
MultinomialNB             29.351555
RandomForestClassifier    22.363115
SVC                       49.971115
Name: accuracy, dtype: float64

In [ ]:
import joblib as jb

jb.dump(models[2],"linearSVC_TTNY.joblib")


In [ ]:
!pip freeze >requirements.txt 